## Librerias

In [2]:
import pymongo
from bson.son import SON
import pandas as pd

# Conexión Base de Datos

In [3]:
client = pymongo.MongoClient('mongodb://localhost:27017')
database = client['Milan_CDR_db']
Milan_CDR_c = database['Milan_CDR_c']

### 1. Encuentra los países con los que se interactúa.

In [4]:
pipeline = [
    {"$group": {"_id": "$countrycode"}},
    {"$sort": SON([("_id", 1)])}
]

distinct_countries = Milan_CDR_c.aggregate(pipeline)
distinct_countries = [doc['_id'] for doc in distinct_countries]
print("Países con los que se interactúa:", distinct_countries)

Países con los que se interactúa: [0, 1, 7, 20, 27, 30, 31, 32, 33, 34, 36, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49, 51, 52, 53, 54, 55, 56, 57, 58, 60, 61, 62, 63, 64, 65, 66, 81, 82, 84, 86, 90, 91, 92, 93, 94, 95, 98, 211, 212, 213, 216, 218, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 291, 297, 298, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 370, 371, 372, 373, 374, 375, 376, 377, 378, 380, 381, 382, 385, 386, 387, 389, 420, 421, 423, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 670, 672, 673, 675, 676, 677, 678, 679, 680, 681, 687, 688, 689, 690, 850, 852, 853, 855, 856, 870, 880, 881, 886, 960, 961, 962, 963, 964, 965, 966, 967, 968, 970, 971, 972, 973, 974, 975, 976, 977, 992, 993, 994, 995, 996, 998, 1139, 1204, 1214, 1226, 1

### 2. Encuentra qué país es con el que más se interactúa además de Italia.


In [6]:
# Italia = 39
# Llamadas internas = 0
excluded_countries = [39, 0]

pipeline_top_countries = [
    {"$match": {"countrycode": {"$nin": excluded_countries}}},
    {"$group": {"_id": "$countrycode", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 1}
]

top_interacted_country = Milan_CDR_c.aggregate(pipeline_top_countries)
top_interacted_country = list(top_interacted_country)[0]['_id']

print("País con el que más se interactúa (además de Italia y otro país excluido):", top_interacted_country)


País con el que más se interactúa (además de Italia y otro país excluido): 49


### 3. ¿Qué celda comunica más con el extranjero?

In [7]:
# Italia = 39
# Llamadas internas = 0
excluded_countries = [39, 0]

pipeline_foreign_cell = [
    {"$match": {"countrycode": {"$nin": excluded_countries}}},
    {"$group": {"_id": "$cellid", "count": {"$sum": 1}}},
    {"$sort": SON([("count", -1)])},
    {"$limit": 1}
]

most_foreign_communicating_cell = Milan_CDR_c.aggregate(pipeline_foreign_cell)
most_foreign_communicating_cell = list(most_foreign_communicating_cell)[0]['_id']
print("Celda que más comunica con el extranjero:", most_foreign_communicating_cell)

Celda que más comunica con el extranjero: 6064


### 4. Encuentra la celda con más actividad de smsin, smsout, callin, callout, internet y la total

In [8]:
# Encuentra la celda con más actividad de smsin
pipeline_smsin = [
    {"$group": {"_id": "$cellid", "total_smsin": {"$sum": "$smsin"}}},
    {"$sort": {"total_smsin": -1}},
    {"$limit": 1}
]

most_active_smsin_cell = Milan_CDR_c.aggregate(pipeline_smsin)
most_active_smsin_cell = list(most_active_smsin_cell)[0]['_id']

# Encuentra la celda con más actividad de smsout
pipeline_smsout = [
    {"$group": {"_id": "$cellid", "total_smsout": {"$sum": "$smsout"}}},
    {"$sort": {"total_smsout": -1}},
    {"$limit": 1}
]

most_active_smsout_cell = Milan_CDR_c.aggregate(pipeline_smsout)
most_active_smsout_cell = list(most_active_smsout_cell)[0]['_id']

# Encuentra la celda con más actividad de callin
pipeline_callin = [
    {"$group": {"_id": "$cellid", "total_callin": {"$sum": "$callin"}}},
    {"$sort": {"total_callin": -1}},
    {"$limit": 1}
]

most_active_callin_cell = Milan_CDR_c.aggregate(pipeline_callin)
most_active_callin_cell = list(most_active_callin_cell)[0]['_id']

# Encuentra la celda con más actividad de callout
pipeline_callout = [
    {"$group": {"_id": "$cellid", "total_callout": {"$sum": "$callout"}}},
    {"$sort": {"total_callout": -1}},
    {"$limit": 1}
]

most_active_callout_cell = Milan_CDR_c.aggregate(pipeline_callout)
most_active_callout_cell = list(most_active_callout_cell)[0]['_id']

# Encuentra la celda con más actividad de Internet
pipeline_internet = [
    {"$group": {"_id": "$cellid", "total_internet": {"$sum": "$Internet"}}},
    {"$sort": {"total_internet": -1}},
    {"$limit": 1}
]

most_active_internet_cell = Milan_CDR_c.aggregate(pipeline_internet)
most_active_internet_cell = list(most_active_internet_cell)[0]['_id']

# Encuentra la celda con más actividad total
pipeline_total_activity = [
    {"$group": {
        "_id": "$cellid",
        "total_activity": {"$sum": {"$sum": ["$smsin", "$smsout", "$callin", "$callout", "$Internet"]}}
    }},
    {"$sort": {"total_activity": -1}},
    {"$limit": 1}
]

most_active_total_cell = Milan_CDR_c.aggregate(pipeline_total_activity)
most_active_total_cell = list(most_active_total_cell)[0]['_id']

# Imprime los resultados
print("Celda con más actividad de smsin:", most_active_smsin_cell)
print("Celda con más actividad de smsout:", most_active_smsout_cell)
print("Celda con más actividad de callin:", most_active_callin_cell)
print("Celda con más actividad de callout:", most_active_callout_cell)
print("Celda con más actividad de Internet:", most_active_internet_cell)
print("Celda con más actividad total:", most_active_total_cell)


Celda con más actividad de smsin: 5161
Celda con más actividad de smsout: 5059
Celda con más actividad de callin: 5059
Celda con más actividad de callout: 5059
Celda con más actividad de Internet: 5161
Celda con más actividad total: 5161


### 5. Crea una colección con un documento por celda en el que aparezcan los acumulados de los diferentes campos.

In [9]:
pipeline_accumulated_by_cell = [
    {"$group": {"_id": "$cellid",
                "total_smsin": {"$sum": "$smsin"},
                "total_smsout": {"$sum": "$smsout"},
                "total_callin": {"$sum": "$callin"},
                "total_callout": {"$sum": "$callout"},
                "total_internet": {"$sum": "$Internet"}}},
    {"$out": "accumulated_by_cell"}
]

Milan_CDR_c.aggregate(pipeline_accumulated_by_cell)

### 6. Crea una colección con un documento por celda y hora en el que aparezcan los acumulados de los diferentes campos.


In [10]:
pipeline_accumulated_by_cell_and_hour = [
    {"$group": {"_id": {"cellid": "$cellid", "hour": {"$hour": {"$toDate": "$time"}}},
                "total_smsin": {"$sum": "$smsin"},
                "total_smsout": {"$sum": "$smsout"},
                "total_callin": {"$sum": "$callin"},
                "total_callout": {"$sum": "$callout"},
                "total_internet": {"$sum": "$Internet"}}},
    {"$out": "accumulated_by_cell_and_hour"}
]

Milan_CDR_c.aggregate(pipeline_accumulated_by_cell_and_hour)

### 7. Realiza un estudio de las celdas especificadas


In [6]:
'''
4259 (Bocconi)
4456 (Navigli)
5060 (Duomo)
1419 (terreno agricula)
2436 (área industrial)
4990 (aeropuerto de Linate)
945 (residencial aislado)
5048 (residencial céntrico)
'''

# Italia = 39
# Llamadas internas = 0
excluded_countries = [39, 0]

selected_cells = [4259, 4456, 5060, 1419, 2436, 4990, 945, 5048]

for cell in selected_cells:
    print("--------------------------------------------------")
    print("         Análisis de la celda:", cell)
    print("--------------------------------------------------\n")

    # CountryCode con los que interactua:
    pipeline_interacted_countries = [
        {"$match": {"cellid": cell}},
        {"$group": {"_id": "$countrycode"}},
        {"$sort": SON([("_id", 1)])}
    ]

    distinct_countries = Milan_CDR_c.aggregate(pipeline_interacted_countries)
    distinct_countries = [doc['_id'] for doc in distinct_countries]
    print("Países con los que se interactúa:", distinct_countries)

    #print("\n")

    # País con el que más se interactúa
    # País con el que más se interactúa
    pipeline_top_countries = [
        {"$match": {"$and": [{"countrycode": {"$nin": excluded_countries}}, {"cellid": cell}]}},
        {"$group": {"_id": "$countrycode", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}},
        {"$limit": 1}
    ]

    top_interacted_country = Milan_CDR_c.aggregate(pipeline_top_countries)
    top_interacted_country = list(top_interacted_country)[0]['_id']

    print("País con el que más se interactúa (además de Italia y llamadas internas):", top_interacted_country)

    #print("\n")

    #Acumulados
    pipeline_cell_analysis = [
        {"$match": {"cellid": cell}},
        {"$group": {"_id": "$cellid",
                    "total_smsin": {"$sum": "$smsin"},
                    "total_smsout": {"$sum": "$smsout"},
                    "total_callin": {"$sum": "$callin"},
                    "total_callout": {"$sum": "$callout"},
                    "total_internet": {"$sum": "$Internet"}}}
    ]

    cell_analysis_result = Milan_CDR_c.aggregate(pipeline_cell_analysis)
    cell_analysis_result = list(cell_analysis_result)[0]
    print(f"Acumulados:")
    print(pd.Series(cell_analysis_result))

    print("\n\n ========================================================================== \n\n")

--------------------------------------------------
         Análisis de la celda: 4259
--------------------------------------------------

Países con los que se interactúa: [0, 1, 7, 20, 27, 30, 31, 32, 33, 34, 36, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49, 51, 52, 53, 54, 55, 57, 60, 61, 62, 63, 64, 66, 81, 82, 84, 86, 90, 91, 94, 98, 212, 213, 216, 220, 221, 226, 228, 230, 234, 242, 244, 251, 256, 291, 351, 352, 353, 355, 356, 357, 358, 359, 370, 371, 372, 373, 377, 380, 381, 382, 385, 386, 420, 421, 503, 505, 507, 591, 593, 852, 880, 886, 961, 965, 966, 971, 972, 974, 975, 995, 1204, 1438, 1514, 1604, 1613, 1780, 7777, 18092, 88239]
País con el que más se interactúa (además de Italia y llamadas internas): 49
Acumulados:
_id                 4259.000000
total_smsin        12724.153334
total_smsout        6468.609191
total_callin        8935.102460
total_callout      10299.850557
total_internet    233453.127646
dtype: float64




--------------------------------------------------
        